In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, cohen_kappa_score, accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score, classification_report


In [13]:

for sub_no in range(1,129):

    data = pd.read_csv(f'Dataset_clean_for_jupyter\\sub-{sub_no}\\eeg\\sub-{sub_no}_majority_yasa_PSG_HB_events.tsv', sep='\t')
    print(sub_no, data['yasa_PSG'].isnull().sum())

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 257
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0


In [15]:
list_of_summaries = []

for sub_no in range(1,129):
    #dropping problematic 20th night
    if sub_no ==20 :
        continue
    data = pd.read_csv(f'Dataset_clean_for_jupyter\\sub-{sub_no}\\eeg\\sub-{sub_no}_majority_yasa_PSG_HB_events.tsv', sep='\t')
    # Calculate simple agreement percentages
    agreement_human_psg = np.mean(data['majority'] == data['yasa_PSG']) * 100
    agreement_human_hb1 = np.mean(data['majority'] == data['HB_1']) * 100
    agreement_human_hb2 = np.mean(data['majority'] == data['HB_2']) * 100
    # Calculate Cohen's Kappa for both devices vs human labels
    kappa_psg = cohen_kappa_score(data['majority'], data['yasa_PSG'])
    kappa_hb1 = cohen_kappa_score(data['majority'], data['HB_1'])
    kappa_hb2 = cohen_kappa_score(data['majority'], data['HB_2'])
    # Analyze how well each device captures transitions between sleep stages
    def analyze_transitions(true_labels, pred_labels):
        transitions_true = sum(1 for i in range(1, len(true_labels)) if true_labels[i] != true_labels[i-1])
        transitions_pred = sum(1 for i in range(1, len(pred_labels)) if pred_labels[i] != pred_labels[i-1])
        
        # Calculate detected transitions
        detected_transitions = 0
        for i in range(1, len(true_labels)):
            if true_labels[i] != true_labels[i-1] and pred_labels[i] != pred_labels[i-1]:
                detected_transitions += 1
        
        transition_recall = detected_transitions / transitions_true if transitions_true > 0 else 0
        
        return {
            'true_transitions': transitions_true,
            'predicted_transitions': transitions_pred,
            'detected_transitions': detected_transitions,
            'transition_recall': transition_recall
        }
    
    psg_transitions = analyze_transitions(data['majority'].values, data['yasa_PSG'].values)
    hb1_transitions = analyze_transitions(data['majority'].values, data['HB_1'].values)
    hb2_transitions = analyze_transitions(data['majority'].values, data['HB_2'].values)

    # Create a summary table of all metrics
    summary = pd.DataFrame({
        'Metric': [
            'Overall Agreement (%)', 
            'Cohen\'s Kappa',
            'Transition Detection Rate'
        ],
        'PSG': [
            agreement_human_psg,
            kappa_psg,
            psg_transitions['transition_recall']
        ],
        'HB_1': [
            agreement_human_hb1,
            kappa_hb1,
            hb1_transitions['transition_recall']
        ],
        'HB_2': [
            agreement_human_hb2,
            kappa_hb2,
            hb2_transitions['transition_recall']
        ]
    })
    
    df_name = f"summary_{sub_no}"
    globals()[df_name] = summary
    list_of_summaries.append(summary)



In [16]:
list_of_summaries[19]

,Metric,PSG,HB_1,HB_2
0,Overall Agreement (%),85.426654,81.783317,83.029722
1,Cohen's Kappa,0.767838,0.735551,0.749160
2,Transition Detection Rate,0.223214,0.375000,0.330357


In [17]:
import pandas as pd
import numpy as np
import os
import glob

# Create a function to combine all summary matrices
def combine_summary_matrices(summary_list=None):
    """
    Combine multiple summary comparison matrices into one final matrix.
    
    Returns:
    --------
    pandas.DataFrame
        Combined summary matrix with average values
    """
    all_summaries = []
    
    # If list of dataframes is provided
    if summary_list is not None:
        all_summaries = summary_list
    else:
        raise ValueError("No summary matrices found to combine")
        
    # Initialize the combined matrix with the same structure
    metrics = all_summaries[0]['Metric'].tolist()
    combined_summary = pd.DataFrame({
        'Metric': metrics,
        'PSG': np.zeros(len(metrics)),
        'HB_1': np.zeros(len(metrics)),
        'HB_2': np.zeros(len(metrics))
    })
    
    # Calculate mean for each metric
    for metric in metrics:
        # Extract values for this metric across all summaries
        psg_values = [df.loc[df['Metric'] == metric, 'PSG'].values[0] for df in all_summaries]
        hb_values = [df.loc[df['Metric'] == metric, 'HB_1'].values[0] for df in all_summaries]
        hb_psg_values = [df.loc[df['Metric'] == metric, 'HB_2'].values[0] for df in all_summaries]

        
        # Calculate mean
        combined_summary.loc[combined_summary['Metric'] == metric, 'PSG'] = np.mean(psg_values)
        combined_summary.loc[combined_summary['Metric'] == metric, 'HB_1'] = np.mean(hb_values)
        combined_summary.loc[combined_summary['Metric'] == metric, 'HB_2'] = np.mean(hb_psg_values)

    
    # Calculate standard deviations
    combined_summary['PSG_STD'] = 0.0
    combined_summary['HB_1_STD'] = 0.0
    combined_summary['HB_2_STD'] = 0.0
    
    
    for metric in metrics:
        psg_values = [df.loc[df['Metric'] == metric, 'PSG'].values[0] for df in all_summaries]
        hb1_values = [df.loc[df['Metric'] == metric, 'HB_1'].values[0] for df in all_summaries]
        hb2_values = [df.loc[df['Metric'] == metric, 'HB_2'].values[0] for df in all_summaries]

        combined_summary.loc[combined_summary['Metric'] == metric, 'PSG_STD'] = np.std(psg_values)
        combined_summary.loc[combined_summary['Metric'] == metric, 'HB_1_STD'] = np.std(hb1_values)
        combined_summary.loc[combined_summary['Metric'] == metric, 'HB_2_STD'] = np.std(hb2_values)
    
    return combined_summary


def format_final_matrix(final_matrix):
    """Format the final matrix for better readability"""
    formatted = final_matrix.copy()

    def format_combined(value, std, is_percentage=False):
        """Formats value and std into a combined string."""
        if is_percentage:
            return f"{value:.2f}% \u00B1 {std:.2f}%"
        else:
            return f"{value:.4f} \u00B1 {std:.4f}"

    # Format percentages for Overall Agreement
    mask_overall = formatted['Metric'] == 'Overall Agreement (%)'
    if not mask_overall.empty:
        formatted.loc[mask_overall, 'PSG'] = formatted.loc[mask_overall, ['PSG', 'PSG_STD']].apply(
            lambda row: format_combined(row['PSG'], row['PSG_STD'], is_percentage=True), axis=1)
        formatted.loc[mask_overall, 'HB_1'] = formatted.loc[mask_overall, ['HB_1', 'HB_1_STD']].apply(
            lambda row: format_combined(row['HB_1'], row['HB_1_STD'], is_percentage=True), axis=1)
        formatted.loc[mask_overall, 'HB_2'] = formatted.loc[mask_overall, ['HB_2', 'HB_2_STD']].apply(
            lambda row: format_combined(row['HB_2'], row['HB_2_STD'], is_percentage=True), axis=1)

    # Format other metrics
    for metric in ['Cohen\'s Kappa', 'Transition Detection Rate']:
        mask_metric = formatted['Metric'] == metric
        if not mask_metric.empty:
            formatted.loc[mask_metric, 'PSG'] = formatted.loc[mask_metric, ['PSG', 'PSG_STD']].apply(
                lambda row: format_combined(row['PSG'], row['PSG_STD']), axis=1)
            formatted.loc[mask_metric, 'HB_1'] = formatted.loc[mask_metric, ['HB_1', 'HB_1_STD']].apply(
                lambda row: format_combined(row['HB_1'], row['HB_1_STD']), axis=1)
            formatted.loc[mask_metric, 'HB_2'] = formatted.loc[mask_metric, ['HB_2', 'HB_2_STD']].apply(
                lambda row: format_combined(row['HB_2'], row['HB_2_STD']), axis=1)

    # Drop the STD columns
    formatted = formatted.drop(columns=['PSG_STD', 'HB_1_STD', 'HB_2_STD'])

    return formatted

In [18]:
final_matrix1 = combine_summary_matrices(summary_list=list_of_summaries)

In [19]:
final_Result = format_final_matrix(final_matrix1)

C:\Users\naikh\AppData\Local\Temp\ipykernel_25372\4059046968.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['77.31% ± 8.77%']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  formatted.loc[mask_overall, 'PSG'] = formatted.loc[mask_overall, ['PSG', 'PSG_STD']].apply(
C:\Users\naikh\AppData\Local\Temp\ipykernel_25372\4059046968.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['69.09% ± 12.54%']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  formatted.loc[mask_overall, 'HB_1'] = formatted.loc[mask_overall, ['HB_1', 'HB_1_STD']].apply(
C:\Users\naikh\AppData\Local\Temp\ipykernel_25372\4059046968.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 

In [20]:
final_Result

,Metric,PSG,HB_1,HB_2
0,Overall Agreement (%),77.31% ± 8.77%,69.09% ± 12.54%,67.64% ± 11.93%
1,Cohen's Kappa,0.6420 ± 0.1277,0.5444 ± 0.1624,0.5218 ± 0.1582
2,Transition Detection Rate,0.3491 ± 0.0874,0.3674 ± 0.1073,0.3493 ± 0.1060
